In [1]:
import numpy as np
import pandas as pd

In [2]:
df_hs = pd.read_csv('Resources/hawaii_stations.csv')
df_hm = pd.read_csv('Resources/hawaii_measurements.csv')
df_hm = df_hm.dropna(axis=0,how='any')
df_hs.to_csv('clean_hawaii_stations.csv',index=False)
df_hm.to_csv('clean_hawaii_measurements.csv',index=False)

In [10]:
df_hm.head()

,station,date,prcp,tobs
0,USC00519397,2010-01-01,0.08,65
1,USC00519397,2010-01-02,0.00,63
2,USC00519397,2010-01-03,0.00,74
3,USC00519397,2010-01-04,0.00,76
5,USC00519397,2010-01-07,0.06,70


In [22]:
df_hs

,station,name,latitude,longitude,elevation
0,USC00519397,"WAIKIKI 717.2, HI US",21.27160,-157.81680,3.0
1,USC00513117,"KANEOHE 838.1, HI US",21.42340,-157.80150,14.6
2,USC00514830,"KUALOA RANCH HEADQUARTERS 886.9, HI US",21.52130,-157.83740,7.0
3,USC00517948,"PEARL CITY, HI US",21.39340,-157.97510,11.9
4,USC00518838,"UPPER WAHIAWA 874.3, HI US",21.49920,-158.01110,306.6
5,USC00519523,"WAIMANALO EXPERIMENTAL FARM, HI US",21.33556,-157.71139,19.5
6,USC00519281,"WAIHEE 837.5, HI US",21.45167,-157.84889,32.9
7,USC00511918,"HONOLULU OBSERVATORY 702.2, HI US",21.31520,-157.99920,0.9
8,USC00516128,"MANOA LYON ARBO 785.2, HI US",21.33310,-157.80250,152.4


In [3]:
import sqlalchemy
from sqlalchemy import create_engine, MetaData
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, Numeric, Text, Float

In [4]:
# Create an engine to a SQLite database file called `hawaii.sqlite`
engine = create_engine("sqlite:///hawaii.sqlite")

In [5]:
# Create a connection to the engine called `conn`
conn = engine.connect()

In [23]:
# Use `declarative_base` and create ORM classes for each table.
# You will need a class for `Measurement` and for `Station`.
# Make sure to define your primary keys.

# http://docs.sqlalchemy.org/en/latest/core/type_basics.html
Base = declarative_base()

class Measurement(Base):
    __tablename__ = 'measurements'

    id = Column(Integer, primary_key=True,autoincrement=True)
    station = Column(Text)
    prcp = Column(Numeric(precision=3))
    tobs = Column(Integer)
    
   
    def __repr__(self):
        return f"id={self.id}, station={self.station}, prcp ={self.prcp}, tops={self.tobs}"

class stations(Base):
    __tablename__ = 'stations'

    station = Column(Text, primary_key=True)
    name = Column(Text)
    latitude = Column(Float)
    longitude = Column(Float)
    elevation = Column(Float)
   
    def __repr__(self):
        return f"id={self.id}, station={self.station}, prcp ={self.prcp}, tops={self.tobs}"

# More on __repr__: https://stackoverflow.com/questions/1984162/purpose-of-pythons-repr 

In [24]:
Base.metadata.create_all(engine)

In [25]:
# For the stations table

# Use Orient='records' to create a list of data to write
# to_dict() cleans out DataFrame metadata as well
# http://pandas-docs.github.io/pandas-docs-travis/io.html#orient-options
data = df_hs.to_dict(orient='records')

In [28]:
# Use MetaData from SQLAlchemy to reflect the tables
metadata = MetaData(bind=engine)
metadata.reflect()
# Save the reference to the `customers` table as a variable called `table`
table = sqlalchemy.Table('stations', metadata, autoload=True)
# Use `table.delete()` to remove any pre-existing data.
# Note that this is a convenience function so that you can re-run the example code multiple times.
# You would not likely do this step in production.
conn.execute(table.delete())

In [29]:
# Use `table.insert()` to insert the data into the table
# The SQL table is populated during this step
conn.execute(table.insert(), data)

In [32]:
# Test that the insert works by fetching the first 5 rows. 
conn.execute("select * from stations limit 10").fetchall()

[('USC00519397', 'WAIKIKI 717.2, HI US', 21.2716, -157.8168, 3.0),
 ('USC00513117', 'KANEOHE 838.1, HI US', 21.4234, -157.8015, 14.6),
 ('USC00514830', 'KUALOA RANCH HEADQUARTERS 886.9, HI US', 21.5213, -157.8374, 7.0),
 ('USC00517948', 'PEARL CITY, HI US', 21.3934, -157.9751, 11.9),
 ('USC00518838', 'UPPER WAHIAWA 874.3, HI US', 21.4992, -158.0111, 306.6),
 ('USC00519523', 'WAIMANALO EXPERIMENTAL FARM, HI US', 21.33556, -157.71139, 19.5),
 ('USC00519281', 'WAIHEE 837.5, HI US', 21.45167, -157.84888999999998, 32.9),
 ('USC00511918', 'HONOLULU OBSERVATORY 702.2, HI US', 21.3152, -157.9992, 0.9),
 ('USC00516128', 'MANOA LYON ARBO 785.2, HI US', 21.3331, -157.8025, 152.4)]

In [33]:
# For the measurements table

# Use Orient='records' to create a list of data to write
# to_dict() cleans out DataFrame metadata as well
# http://pandas-docs.github.io/pandas-docs-travis/io.html#orient-options
data = df_hm.to_dict(orient='records')

In [34]:
# Use MetaData from SQLAlchemy to reflect the tables
metadata = MetaData(bind=engine)
metadata.reflect()
# Save the reference to the `measurements` table as a variable called `table`
table = sqlalchemy.Table('measurements', metadata, autoload=True)
# Use `table.delete()` to remove any pre-existing data.
# Note that this is a convenience function so that you can re-run the example code multiple times.
# You would not likely do this step in production.
conn.execute(table.delete())

In [35]:
# Use `table.insert()` to insert the data into the table
# The SQL table is populated during this step
conn.execute(table.insert(), data)

In [36]:
# Test that the insert works by fetching the first 5 rows. 
conn.execute("select * from measurements limit 100").fetchall()

[(1, 'USC00519397', 0.08, 65),
 (2, 'USC00519397', 0, 63),
 (3, 'USC00519397', 0, 74),
 (4, 'USC00519397', 0, 76),
 (5, 'USC00519397', 0.06, 70),
 (6, 'USC00519397', 0, 64),
 (7, 'USC00519397', 0, 68),
 (8, 'USC00519397', 0, 73),
 (9, 'USC00519397', 0.01, 64),
 (10, 'USC00519397', 0, 61),
 (11, 'USC00519397', 0, 66),
 (12, 'USC00519397', 0, 65),
 (13, 'USC00519397', 0, 68),
 (14, 'USC00519397', 0, 64),
 (15, 'USC00519397', 0, 72),
 (16, 'USC00519397', 0, 66),
 (17, 'USC00519397', 0, 66),
 (18, 'USC00519397', 0, 69),
 (19, 'USC00519397', 0, 67),
 (20, 'USC00519397', 0, 67),
 (21, 'USC00519397', 0.01, 71),
 (22, 'USC00519397', 0, 67),
 (23, 'USC00519397', 0.04, 76),
 (24, 'USC00519397', 0.12, 68),
 (25, 'USC00519397', 0, 72),
 (26, 'USC00519397', 0.03, 67),
 (27, 'USC00519397', 0.01, 66),
 (28, 'USC00519397', 0.01, 69),
 (29, 'USC00519397', 0, 67),
 (30, 'USC00519397', 0, 67),
 (31, 'USC00519397', 0, 64),
 (32, 'USC00519397', 0, 69),
 (33, 'USC00519397', 0, 73),
 (34, 'USC00519397', 0, 7

In [40]:
conn.close()